## Legs
Los objetos de tipo `Leg` son una lista (o vector) de objetos `Cashflow` y representan una pata de un instrumento financiero. un objeto de tipo `Leg` puede construirse *a mano* es decir, dando de alta cashflows y agregándolos uno a uno o con algunos métodos de conveniencia cuyo funcionamiento se mostrará.
### Construcción Manual
Se verá como construir un `Leg` con 2 `SimpleCashflow` de forma *manual*. En particular, este objeto `Leg` podría representar una operación FX por entrega física.

In [1]:
import QC_Financial as qcf

In [2]:
leg = qcf.Leg()
fecha_vcto = qcf.QCDate(20, 9, 2018)
simple_cashflow_1 = qcf.SimpleCashflow(fecha_vcto,  # fecha del flujo
                                       100,         # monto
                                       qcf.QCCLP()) # moneda

simple_cashflow_2 = qcf.SimpleCashflow(fecha_vcto,  # fecha del flujo
                                       -70000,      # monto
                                       qcf.QCUSD()) # moneda
leg.append_cashflow(simple_cashflow_1)
leg.append_cashflow(simple_cashflow_2)

In [3]:
# Se observa el resultado
num = leg.size()
for i in range(0, num):
    print qcf.show(leg.get_cashflow_at(i))

('2018-09-20', 100.0, 'CLP')
('2018-09-20', -70000.0, 'USD')


### Construcción Asistida de un `FixedRateLeg`
Se verá como construir objetos `Leg` donde cada `Cashflow` es un objeto de tipo `FixedRateCashflow`, todos con la misma tasa fija. En el primer ejemplo se construye un `Leg` de tipo *bullet*: una única amortización igual al capital vigente de todos los `FixedRateCasflow` en el último flujo.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal inicial
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámteros siguientes se puede visualizar el efecto de ellos en la construcción.

In [4]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(12, 1, 2018)
fecha_final = qcf.QCDate(12, 3, 2023) 
bus_adj_rule = qcf.BusyAdjRules.FOLLOW
periodicidad = qcf.Tenor('6M')
periodo_irregular = qcf.StubPeriod.SHORTBACK
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 100000.0
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.03, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

# Se da de alta el objeto
fixed_rate_leg = qcf.LegFactory.build_bullet_fixed_rate_leg(rp,
                                                            fecha_inicio,
                                                            fecha_final,
                                                            bus_adj_rule,
                                                            periodicidad,
                                                            periodo_irregular,
                                                            calendario,
                                                            lag_pago,
                                                            nominal,
                                                            amort_es_flujo,
                                                            tasa_cupon,
                                                            moneda,
                                                            es_bono)

Se puede lograr una visualización mucho mejor del resultado utilizando un Dataframe de pandas.

In [5]:
# Se importa pandas de la forma usual
import pandas as pd

# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_leg.size()):
    tabla.append(qcf.show(fixed_rate_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2018-01-12,2018-07-12,2018-07-12,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
1,2018-07-12,2019-01-14,2019-01-14,100000.0,0.0,1550.000000,False,1550.000000,CLF,0.03,LinAct360
2,2019-01-14,2019-07-12,2019-07-12,100000.0,0.0,1491.666667,False,1491.666667,CLF,0.03,LinAct360
3,2019-07-12,2020-01-13,2020-01-13,100000.0,0.0,1541.666667,False,1541.666667,CLF,0.03,LinAct360
4,2020-01-13,2020-07-13,2020-07-13,100000.0,0.0,1516.666667,False,1516.666667,CLF,0.03,LinAct360
5,2020-07-13,2021-01-12,2021-01-12,100000.0,0.0,1525.000000,False,1525.000000,CLF,0.03,LinAct360
6,2021-01-12,2021-07-12,2021-07-12,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
7,2021-07-12,2022-01-12,2022-01-12,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360
8,2022-01-12,2022-07-12,2022-07-12,100000.0,0.0,1508.333333,False,1508.333333,CLF,0.03,LinAct360
9,2022-07-12,2023-01-12,2023-01-12,100000.0,0.0,1533.333333,False,1533.333333,CLF,0.03,LinAct360


### Construcción Asistida de un `CustomAmortFixedRateLeg`
En este ejemplo se construye un `Leg` donde la estructura de amortizaciones es customizada.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `CustomNotionalAndAmort`: vector de capital vigente y amortizaciones customizado
- `bool`: si es `True` significa que la amortización es un flujo de caja efectivo
- `QCInterestRate`: la tasa a aplicar en cada flujo
- `QCCurrency`: moneda del nominal y de los flujos
- `bool`: si es `True` fuerza a que las fechas de pago coincidan con las fechas finales. Esto para lograr una valorización acorde a las convenciones de los mercados de renta fija, en caso que la `Leg` represente un bono a tasa fija.

Vamos a un ejemplo. Cambiando los parámteros siguientes se puede visualizar el efecto de ellos en la construcción.

In [22]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(8, 3, 2018)
fecha_final = qcf.QCDate(8, 9, 2033) 
bus_adj_rule = qcf.BusyAdjRules.NO
periodicidad = qcf.Tenor('1M')
periodo_irregular = qcf.StubPeriod.LONGFRONT6
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
custom_notional_amort = qcf.CustomNotionalAmort()
custom_notional_amort.set_size(10)
for i in range(0, 10):
    custom_notional_amort.set_notional_amort_at(i, 1000.0 - i * 100.0, 100.0)
amort_es_flujo = False
tasa_cupon = qcf.QCInterestRate(.0479, qcf.QCAct360(), qcf.QCLinearWf())
moneda = qcf.QCCLF()
es_bono = False

In [23]:
# Se da de alta el objeto
fixed_rate_custom_leg = qcf.LegFactory.build_custom_amort_fixed_rate_leg(rp,
                                                                         fecha_inicio,
                                                                         fecha_final,
                                                                         bus_adj_rule,
                                                                         periodicidad,
                                                                         periodo_irregular,
                                                                         calendario,
                                                                         lag_pago,
                                                                         custom_notional_amort,
                                                                         amort_es_flujo,
                                                                         tasa_cupon,
                                                                         moneda)

In [24]:
# Se importa pandas de la forma usual
import pandas as pd

# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, fixed_rate_custom_leg.size()):
    tabla.append(qcf.show(fixed_rate_custom_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo', 'moneda',
            'valor_tasa', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df

,fecha_ini,fecha_fin,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,valor_tasa,tipo_tasa
0,2018-03-08,2018-09-08,2018-09-10,1.0,0.0,0.024482,False,0.024482,CLF,0.0479,LinAct360
1,2018-09-08,2018-10-08,2018-10-08,1.0,0.0,0.003992,False,0.003992,CLF,0.0479,LinAct360
2,2018-10-08,2018-11-08,2018-11-08,1.0,0.0,0.004125,False,0.004125,CLF,0.0479,LinAct360
3,2018-11-08,2018-12-08,2018-12-10,1.0,0.0,0.003992,False,0.003992,CLF,0.0479,LinAct360
4,2018-12-08,2019-01-08,2019-01-08,1.0,0.0,0.004125,False,0.004125,CLF,0.0479,LinAct360
5,2019-01-08,2019-02-08,2019-02-08,1.0,0.0,0.004125,False,0.004125,CLF,0.0479,LinAct360
6,2019-02-08,2019-03-08,2019-03-08,1.0,0.0,0.003726,False,0.003726,CLF,0.0479,LinAct360
7,2019-03-08,2019-04-08,2019-04-08,1.0,0.0,0.004125,False,0.004125,CLF,0.0479,LinAct360
8,2019-04-08,2019-05-08,2019-05-08,1.0,0.0,0.003992,False,0.003992,CLF,0.0479,LinAct360
9,2019-05-08,2019-06-08,2019-06-10,1.0,0.0,0.004125,False,0.004125,CLF,0.0479,LinAct360


### Construcción Asistida de un `BulletIborLeg`
En este ejemplo se construye un `Leg` con `IborCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `Tenor`: periodicidad de fijación
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular para el calendario de fijaciones
- `QCBusinessCalendar`: calendario que aplica para las fechas de fijación
- `unsigned int`: lag de fijación expresado en días
- `InterestRateIndex`: índice de tasa de interés utilizado en cada `IborCashflow`
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `QCCurrency`: moneda del nominal y de los flujos
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IborCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_ibor_leg(...)`.

In [9]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
periodicidad_fijacion = qcf.Tenor('3M')
periodo_irregular_fijacion = qcf.StubPeriod.NO
# vamos a usar el mismo calendario para pago y fijaciones
lag_de_fijacion = 2

# Definición del índice
codigo = 'LIBORUSD3M'
lin_act360 = qcf.QCInterestRate(.0, qcf.QCAct360(), qcf.QCLinearWf())
fixing_lag = qcf.Tenor('2d')
tenor = qcf.Tenor('3m')
fixing_calendar = calendario
settlement_calendar = calendario
usd = qcf.QCUSD()
libor_usd_3m = qcf.InterestRateIndex(codigo,
                                    lin_act360,
                                    fixing_lag,
                                    tenor,
                                    fixing_calendar,
                                    settlement_calendar,
                                    usd)
# Fin índice
nominal = 1000000.0
amort_es_flujo = True 
moneda = usd
spread = .01
gearing = 1.0

ibor_leg = qcf.LegFactory.build_bullet_ibor_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     periodicidad_fijacion, periodo_irregular_fijacion,
                                                     calendario, lag_de_fijacion, libor_usd_3m,
                                                     nominal, amort_es_flujo, moneda, spread, gearing)

In [11]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_leg.size()):
    tabla.append(qcf.show(ibor_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_fix', 'fecha_pago', 'nominal', 'amort', 'interes', 'amort_es_flujo', 'flujo',
            'moneda', 'codigo_indice', 'valor_tasa', 'spread', 'gearing', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df

,fecha_ini,fecha_fin,fecha_fix,fecha_pago,nominal,amort,interes,amort_es_flujo,flujo,moneda,codigo_indice,valor_tasa,spread,gearing,tipo_tasa
0,1969-01-31,1969-04-30,1969-01-29,1969-04-30,1000000.0,0.0,2472.222222,True,2.472222e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
1,1969-04-30,1969-07-31,1969-04-28,1969-07-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
2,1969-07-31,1969-10-31,1969-07-29,1969-10-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
3,1969-10-31,1970-01-30,1969-10-29,1970-01-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
4,1970-01-30,1970-04-30,1970-01-28,1970-04-30,1000000.0,0.0,2500.000000,True,2.500000e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
5,1970-04-30,1970-07-31,1970-04-28,1970-07-31,1000000.0,0.0,2555.555556,True,2.555556e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
6,1970-07-31,1970-10-30,1970-07-29,1970-10-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
7,1970-10-30,1971-01-29,1970-10-28,1971-01-29,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
8,1971-01-29,1971-04-30,1971-01-27,1971-04-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360
9,1971-04-30,1971-07-30,1971-04-28,1971-07-30,1000000.0,0.0,2527.777778,True,2.527778e+03,USD,LIBORUSD3M,0.0,0.01,1.0,LinAct360


### Construcción Asistida de un `IcpClpLeg`
En este ejemplo se construye un `Leg` con `IcpClpCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClpCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clp_leg(...)`.

In [ ]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0

icp_clp_leg = qcf.LegFactory.build_bullet_icp_clp_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     nominal, amort_es_flujo, spread, gearing)

In [ ]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_cashflow.size()):
    tabla.append(qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df

### Construcción Asistida de un `IcpClfLeg`
En este ejemplo se construye un `Leg` con `IcpClfCashflow` y amortización bullet.
Se requieren los siguientes parámetros:

- `RecPay`: enum que indica si los flujos se reciben o pagan
- `QCDate`: fecha de inicio del primer flujo
- `QCDate`: fecha final del último flujo sin considerar ajustes de días feriados
- `BusyAdRules`: enum que representa el tipo de ajuste en la fecha final para días feriados
- `Tenor`: la periodicidad de pago
- `QCInterestRateLeg::QCStubPeriod`: enum que representa el tipo de período irregular (si aplica)
- `QCBusinessCalendar`: calendario que aplica para las fechas de pago
- `unsigned int`: lag de pago expresado en días
- `float`: nominal
- `bool`: si es `True` significa que la amortización final es un flujo de caja efectivo
- `float`: spread aditivo
- `gearing`: spread multiplicativo

Vamos a un ejemplo. Cambiando los parámetros anteriores se puede visualizar el efecto de ellos en la construcción. 

**NOTA:** para construir un `Leg` con `IcpClfCashflow` y amortización customizada, sólo se debe cambiar el parámetro **nominal** por **CustomNotionalAndAmort** e invocar el método `qcf.LegFactory.build_custom_amort_icp_clf_leg(...)`.

In [ ]:
# Se da de alta los parámetros requeridos
rp = qcf.RecPay.RECEIVE
fecha_inicio = qcf.QCDate(31, 1, 1969)
fecha_final = qcf.QCDate(31, 1, 1974) 
bus_adj_rule = qcf.BusyAdjRules.MODFOLLOW
periodicidad_pago = qcf.Tenor('3M')
periodo_irregular_pago = qcf.StubPeriod.NO
calendario = qcf.BusinessCalendar(fecha_inicio, 20)
lag_pago = 0
nominal = 1000000.0
amort_es_flujo = True 
spread = .01
gearing = 1.0

icp_clp_leg = qcf.LegFactory.build_bullet_icp_clf_leg(rp, fecha_inicio, fecha_final, bus_adj_rule, periodicidad_pago,
                                                     periodo_irregular_pago, calendario, lag_pago,
                                                     nominal, amort_es_flujo, spread, gearing)

In [ ]:
# Se define un list donde almacenar los resultados de la función show
tabla = []
for i in range(0, ibor_cashflow.size()):
    tabla.append(qcf.show(icp_clp_leg.get_cashflow_at(i)))

# Se utiliza tabla para inicializar el Dataframe
columnas = ['fecha_ini', 'fecha_fin', 'fecha_pago', 'nominal', 'amort', 'amort_es_flujo', 'flujo',
            'moneda', 'icp_inicial', 'icp_final', 'uf_inicial', 'uf_final',
            'valor_tasa', 'interes', 'spread', 'gearing', 'tipo_tasa']
df = pd.DataFrame(tabla, columns=columnas)

# Se despliega la data en este formato
df